# DB2-Salesforce connector: Tool usage clusters information updates

In [1]:
# API settings
api_url = '/services/data/v43.0/sobjects'
external_id = 'ID__c'
object_id = 'tool_usage_cluster__c'

# login parameters to be handled by Papermill
sql_login_params = {"username": "wang2506_ro", "password": "fnVnwcCS7iT45EsA"}
sf_login_params = {
    "grant_type": "password",
    "client_id": "3MVG95jctIhbyCppj0SNJ75IsZ1y8UPGZtSNF4j8FNVXz.De8Lu4jHm3rjRosAtsHy6qjHx3i4S_QbQzvBePG",
    "client_secret": "D1623C6D3607D4FC8004B92C761DFB6C1F70CCD129C5501E357028DFA00F5764",
    "username":"wang2506@purdue.edu",
    "password":"purdue99272844vp8Z6qq0teuPQmirGaaPAt4",   
} 
    #"username": "wang159-4j1v@force.com",
    #"password": "napoleon0eZ3PQpQqE3C3z4wWqEqKGhQ8",

In [2]:
import sys
sys.path.append('/home/users/wang2506/nanohub_salesforce_integ/salesforce')
    
import pandas as pd
import datetime

## Obtain cluster information from DB2 

In [3]:
# connect with DB2
import sqlalchemy as sql

engine = sql.create_engine('mysql+pymysql://%s:%s@127.0.0.1/wang159_myrmekes' \
                                               %(sql_login_params['username'], sql_login_params['password']))

In [4]:
class_info_df = pd.read_sql_query('select * from cluster_class_info', engine)
#students_info_df = pd.read_sql_query('select * from cluster_students_info', engine)
#classtool_info_df = pd.read_sql_query('select * from cluster_classtool_info', engine)

# display
display(class_info_df.head(2))
#display(students_info_df.head(2))
#display(classtool_info_df.head(2))

,index,class_id,start,end,lon,lat,size
0,0,0,2013-09-13,2013-09-20,-86.907866,40.430972,93
1,1,1,2013-09-15,2013-09-20,-87.924079,42.019433,10


In [5]:
# convert and clean some columns, so we can, for example, sort values
class_info_df['start'] = pd.to_datetime(class_info_df['start'], errors='coerce') 
class_info_df['end'] = pd.to_datetime(class_info_df['end'], errors='coerce')

In [6]:
# create geolocation Estimated_Location__c
# {"house":{"location__latitude__s":34, "location__longitude__s":23}}

class_info_df['Estimated_Location__c'] = class_info_df.apply(lambda x: {"Estimated_Location__c":\
                                                                 {"location__latitude__s":x.lat, "location__longitude__s":x.lon}}, axis=1)

class_info_df['Map_Location__c'] = class_info_df\
      .apply(lambda x: 'https://www.google.com/maps/@?api=1&map_action=map&center=%f,%f&zoom=9'%(x.lat, x.lon), axis=1)

In [7]:
class_info_df.head(2)

,index,class_id,start,end,lon,lat,size,Estimated_Location__c,Map_Location__c
0,0,0,2013-09-13,2013-09-20,-86.907866,40.430972,93,{'Estimated_Location__c': {'location__latitude...,https://www.google.com/maps/@?api=1&map_action...
1,1,1,2013-09-15,2013-09-20,-87.924079,42.019433,10,{'Estimated_Location__c': {'location__latitude...,https://www.google.com/maps/@?api=1&map_action...


## Match data with Salesforce format

In [8]:
df_sf = pd.DataFrame()

# Make sure NaN and NaT values are taken care of here

df_sf['ID__c']                             = class_info_df['class_id']
df_sf['Starting_Date__c']          = class_info_df['start']
df_sf['Ending_Date__c']            = class_info_df['end']
df_sf['Map_Location__c'] = class_info_df['Map_Location__c']
df_sf['Estimated_Location__latitude__s'] = class_info_df['lat']
df_sf['Estimated_Location__longitude__s'] = class_info_df['lon']

sf_original_fields = df_sf.columns

# display
df_sf.head(2).T

,0,1
ID__c,0,1
Starting_Date__c,2013-09-13 00:00:00,2013-09-15 00:00:00
Ending_Date__c,2013-09-20 00:00:00,2013-09-20 00:00:00
Map_Location__c,https://www.google.com/maps/@?api=1&map_action...,https://www.google.com/maps/@?api=1&map_action...
Estimated_Location__latitude__s,40.431,42.0194
Estimated_Location__longitude__s,-86.9079,-87.9241


## To Salesforce Sales Cloud CRM

In [9]:
from DB2SalesforceAPI import DB2SalesforceAPI

# create DB2 to Salesforce API object
db_s = DB2SalesforceAPI(sf_login_params)

# specify Salesforce object ID and external ID
db_s.object_id = object_id
db_s.external_id = external_id

Obtained Salesforce access token ...... True


In [10]:
# send data to Salesforce
db_s.send_data(df_sf)

[Success] Bulk job creation successful. Job ID = 7505w00000TmOrQAAV
hello


[Success] CSV upload successful. Job ID = 7505w00000TmOrQAAV
[Success] Closing job successful. Job ID = 7505w00000TmOrQAAV


In [11]:
# check status
db_s.check_bulk_status()

{'id': '7505w00000TmOrQAAV',
 'operation': 'upsert',
 'object': 'tool_usage_cluster__c',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2021-01-21T17:07:17.000+0000',
 'systemModstamp': '2021-01-21T17:07:18.000+0000',
 'state': 'UploadComplete',
 'externalIdFieldName': 'ID__c',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 0,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apexProcessingTime': 0}

In [12]:
from pprint import pprint
pprint(db_s.check_bulk_failed_results())

''
